## Importing the Libraries

In [2]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
import pickle
from sklearn.model_selection import GridSearchCV, KFold

Using TensorFlow backend.


## Reading the images from the folders and assigning to Training and validation

In [3]:
train_datagen = ImageDataGenerator(rescale=1. / 255)
itr = train_generator = train_datagen.flow_from_directory('train',
                                                          target_size=(128, 128),
                                                          batch_size=3778,
                                                          class_mode='categorical')

X_train, y_train = itr.next()

val_datagen = ImageDataGenerator(rescale=1. / 255)
itr = train_generator = val_datagen.flow_from_directory('valid',
                                                          target_size=(128, 128),
                                                        batch_size=951,
                                                          class_mode='categorical')

X_val, y_val = itr.next()

Found 2682 images belonging to 7 classes.
Found 676 images belonging to 7 classes.


## Extracting the target variable & Flattening the images

In [4]:
Y_train = []
for y in list(y_train):
    Y_train.append(np.argmax(y))
    
Y_val = []
for y in list(y_val):
    Y_val.append(np.argmax(y))
    

def flat(X):
    X_data = []
    for x in X:
        X_data.append(x.flatten())
    return X_data

X_train = flat(X_train)
X_val = flat(X_val)

## Training the model

In [5]:
svc = SVC(verbose=True)
svc.fit(X_train,Y_train)

/Users/ravidahiya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)

## Saving the model

In [7]:
from sklearn.externals import joblib
joblib.dump(svc, 'model.pkl')

['model.pkl']

## Model Performance

In [14]:
pred = model.predict(X_val)
print(classification_report(Y_val,pred))

              precision    recall  f1-score   support

           0       0.45      0.57      0.50       113
           1       0.50      0.06      0.10        53
           2       0.40      0.41      0.41        87
           3       0.37      0.48      0.42       125
           4       0.50      0.58      0.54        89
           5       0.36      0.25      0.29       113
           6       0.36      0.35      0.36        96

    accuracy                           0.41       676
   macro avg       0.42      0.39      0.37       676
weighted avg       0.41      0.41      0.39       676



## Tuning the model

In [6]:
svc = SVC()
k_fold = KFold(5,random_state=33)
params = {'C':[0.01,0.001],'kernel':['linear'],'max_iter':[100,1000]}
model = GridSearchCV(estimator=svc,param_grid=params,scoring='accuracy',cv=k_fold)

/Users/ravidahiya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [7]:
model.fit(X_train,Y_train)

/Users/ravidahiya/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ravidahiya/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ravidahiya/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ravidahiya/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, Converge

GridSearchCV(cv=KFold(n_splits=5, random_state=33, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.01, 0.001], 'kernel': ['linear'],
                         'max_iter': [100, 1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [10]:
pred = model.best_estimator_.predict(X_val)
print(classification_report(Y_val,pred))

              precision    recall  f1-score   support

           0       0.37      0.52      0.43       113
           1       0.20      0.11      0.14        53
           2       0.37      0.49      0.42        87
           3       0.39      0.40      0.40       125
           4       0.57      0.43      0.49        89
           5       0.34      0.30      0.32       113
           6       0.33      0.26      0.29        96

    accuracy                           0.38       676
   macro avg       0.37      0.36      0.36       676
weighted avg       0.38      0.38      0.37       676



### As we can see that model tuning didn't improve the results, we will use default model for predictions